In [2]:
from cmath import nan
import numpy as np
import pandas as pd
from sqlalchemy import null

def whfj_anlysis(x):
    # 单位为价格待定，则返回NaN
    if x in ['价格待定']:
        return np.NaN
    else:
        return x
    
# 如果价格为万元，则修改单位为元/㎡，否则返回原单位
# 单价为万元，则单价乘以10000
def whfj_anlysis_price(x):
    if x in ['万元']:
        return '元/㎡'
    else:
        return x

df = pd.read_csv('data/whftx.csv')
df.rename(columns={'标题':'楼盘名称', 'house_value': 'type', '类型':'房型', 'sngrey':'area', 'insale':'insale', 'fangyuan':'房源',
                   '价格': '单价','价格5':'单位','价格6':'总价'}, inplace=True)
df['楼盘名称'] = df.楼盘名称.str.strip('\n\t\t\t\t\t')
df['房型'] = df.房型.apply(lambda x: x.split('/',1)[0].strip('\n\t\t\t\t\t'))
df['房型'] = df.房型.apply(lambda x: x.split('\n',1)[0].strip('\n\t\t\t\t\t'))
df['地址'] = df.地址.apply(lambda x: x.split(']',1)[0].strip('\n\t\t\t\t\t'))
df['地址'] = df.地址.apply(lambda x: x.split('[',1)[-1].strip())
df['area'] = df.area.apply(lambda x: x.split(']',1)[0].strip('\n\t\t\t\t\t'))
df['area'] = df.area.apply(lambda x: x.split('[',1)[-1].strip())
df['insale'] = df.insale.str.strip('\n\t\t\t\t\t')
df['单价'] = df.单价.apply(whfj_anlysis)
df['单价'] = np.where(df['单位']=='万元/㎡', df['单价'] * 10000, df['单价'])
# 处理单价单位
df['单位'] = df.单位.apply(whfj_anlysis_price)
df['单位'] = df.单位.astype('str').apply(lambda x: x.split('(',1)[0].strip())
df['单位'] = df.单位.apply(lambda x: x.split('起',1)[0].strip())
df['总价'] = df.总价.astype('str').apply(lambda x: x.split('/套',1)[0].strip())
df1 = df[['楼盘名称','type','房型','area','insale','房源','单价','单位','总价']]
df1.to_csv('data/whfj_anlysis.csv', index=False)
df1.loc[(df1['单位'] == '万元/套'),'总价'] = df['单价']+'万元'
df1.loc[(df1['单位'] == '万元/套'),'单价'] = nan
df1.loc[(df1['单位'] == '万元/套'),'单位'] = nan
df1.to_csv('data/whfj_anlysis_new.csv', index=False)
df2 = pd.read_csv('data/whfj_anlysis_new.csv')
df2


,楼盘名称,type,房型,area,insale,房源,单价,单位,总价
0,天创长江云栖风华,直播中,三居,汉阳,NaN,普通住宅,NaN,NaN,NaN
1,龙湖·揽境,真现房,三居,硚口,在售,普通住宅,13500.0,元/㎡,168万元
2,武汉沙湖中心·云玺,NaN,NaN,武昌,在售,地铁沿线,13000.0,元/㎡,NaN
3,中建壹品·汉芯公馆,纯新盘,三居,江岸,NaN,普通住宅,NaN,NaN,NaN
4,保利天珺,NaN,三居,江岸,在售,普通住宅,45000.0,元/㎡,520万元
...,...,...,...,...,...,...,...,...,...
1183,信达地产天纵第五园,NaN,二居,硚口,在售,普通住宅,15800.0,元/㎡,128万元
1184,天创黄金时代,NaN,三居,汉阳,在售,普通住宅,11000.0,元/㎡,104万元
1185,融创光谷壹号院,NaN,三居,东湖高新区,在售,普通住宅,29500.0,元/㎡,310万元
1186,尚城国际,NaN,三居,汉阳,在售,普通住宅,16000.0,元/㎡,170万元


In [9]:
# 1. 分析在售房数量
from pyparsing import nums


insales = df2[df2.insale == '在售']
insales.insale.count()
# 2. 楼盘数量
# df2.groupby('楼盘名称')['楼盘名称'].to_list()
# 3. 不同地区不同房型的在售房数量
df3 = df2[df2['insale']=='在售'].groupby(['area', '房型'])['insale'].count().index.to_list()
numsnums = df2[df2['insale']=='在售'].groupby(['area', '房型'])['insale'].count().values.tolist()
print(df3)
print(nums)

# 4. 不同地区不同房型的新房数量
xf = df2[df2['type']=='纯新盘'].groupby(['area', '房型'])['type'].count().index.to_list()
xfnums = df2[df2['type']=='纯新盘'].groupby(['area', '房型'])['type'].count().values.tolist()
print(xf)
print(xfnums)
# 5. 不同地区不同房型的在售单价排行榜
insale_new_house = df2.query('insale=="在售"').groupby(['area', '房型'])['单价'].sum().sort_values(ascending=False).index.to_list()
insale_new_house_nums = df2.query('insale=="在售"').groupby(['area', '房型'])['单价'].sum().sort_values(ascending=False).values.tolist()
print(insale_new_house)
print(insale_new_house_nums)
# 6. 不同地区不同房型的在售总价排行榜
area = df2['area'].drop_duplicates().tolist()
chamber = df2['房型'].drop_duplicates().tolist()
sum_price = df2['总价'].str.strip('万元').tolist()
print(area)
print(chamber)
print(sum_price)


[('东湖高新区', '三居'), ('东西湖', '三居'), ('东西湖', '二居'), ('汉阳', '三居'), ('江夏', '三居'), ('江岸', '三居'), ('江岸', '四居'), ('江汉', '三居'), ('洪山', '三居'), ('洪山', '四居'), ('硚口', '三居'), ('硚口', '二居'), ('经济开发区', '三居'), ('蔡甸', '二居'), ('青山', '三居')]
0123456789
[('东湖高新区', '三居'), ('江岸', '三居'), ('洪山', '三居')]
[1, 33, 33]
[('江岸', '三居'), ('洪山', '三居'), ('汉阳', '三居'), ('硚口', '三居'), ('江汉', '三居'), ('青山', '三居'), ('江夏', '三居'), ('洪山', '四居'), ('江岸', '四居'), ('东湖高新区', '三居'), ('硚口', '二居'), ('东西湖', '二居'), ('经济开发区', '三居'), ('东西湖', '三居'), ('蔡甸', '二居')]
[3934227.0, 3226300.0, 2046000.0, 2046000.0, 1485000.0, 1435500.0, 1339800.0, 1320000.0, 1188000.0, 993500.0, 521400.0, 462000.0, 445500.0, 346500.0, 260700.0]
['汉阳', '硚口', '武昌', '江岸', '东湖高新区', '青山', '洪山', '经济开发区', '东西湖', '江夏', '江汉', '蔡甸']
['三居', nan, '四居', '二居']
[nan, '168', nan, nan, '520', '196', '218', '129', '504', '384', '350', '101', nan, '83', '149', '246', '113', '160', nan, '116', '138', nan, nan, nan, nan, nan, nan, '350', nan, '204', '62', '128', '104', '310', '170', '94', nan

In [10]:
#  不同地区不同房型的新房数量, 柱形图
# 分组统计
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker


xf = df2[df2['type']=='纯新盘'].groupby(['area', '房型'])['type'].count().index.to_list()
xfnums = df2[df2['type']=='纯新盘'].groupby(['area', '房型'])['type'].count().values.tolist()

(
    Bar()
   .add_xaxis(xf)
   .add_yaxis('各个地区各个房型的纯新盘分析图', xfnums)
   .set_global_opts(title_opts=opts.TitleOpts(title='各个地区各个房型的纯新盘分析图', subtitle='各个地区各个房型的纯新盘数量'))
).render_notebook()


In [11]:
#  不同地区不同房型的在售房数量, 柱形图
# 分组统计
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker


df3 = df2[df2['insale']=='在售'].groupby(['area', '房型'])['insale'].count().index.to_list()
nums= df2[df2['insale']=='在售'].groupby(['area', '房型'])['insale'].count().values.tolist()

(
    Bar()
   .add_xaxis(df3)
   .add_yaxis('各个地区各个房型的在售房分析图', nums)
   .set_global_opts(title_opts=opts.TitleOpts(title='各个地区各个房型的在售房分析图', subtitle='各个地区各个房型的在售房数量'))
).render_notebook()

In [12]:
from pyecharts import options as opts
from pyecharts.charts import PictorialBar
from pyecharts.globals import SymbolType

location = df2.query('insale=="在售"').groupby(['area', '房型'])['单价'].sum().sort_values(ascending=False).index.to_list()
values = df2.query('insale=="在售"').groupby(['area', '房型'])['单价'].sum().sort_values(ascending=False).values.tolist()

c = (
    PictorialBar()
    .add_xaxis(location)
    .add_yaxis(
        "",
        values,
        label_opts=opts.LabelOpts(is_show=False),
        symbol_size=18,
        symbol_repeat="fixed",
        symbol_offset=[0, 0],
        is_symbol_clip=True,
        symbol=SymbolType.ROUND_RECT,
    )
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title="PictorialBar-各地区各房型单价排行榜"),
        xaxis_opts=opts.AxisOpts(is_show=False),
        yaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_show=False),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(opacity=0)
            ),
        ),
    )
    .render("pictorialbar_base.html")
)

In [2]:
from ast import If
from tkinter import N
from numpy import NaN
import pyecharts.options as opts
from pyecharts.charts import Bar3D
import pandas as pd

df2 = pd.read_csv('data/whfj_anlysis_new.csv')
# print(df2)

area = df2['area'].drop_duplicates().tolist()
chamber = df2['房型'].drop_duplicates().tolist()
sum_price = df2['总价'].str.strip('万元').tolist()

def get_area_index(area_name):
    for i in range(len(area)):
        if area[i] == area_name:
            return i

def get_chamber_index(chamber_name):
    for i in range(len(chamber)):
        if chamber[i] == chamber_name:
            return i

# print(df2['房型'].apply(get_chamber_index))
# .apply(get_area_index)

s = pd.Series(df2['房型'].apply(get_chamber_index).to_list()) # 转换为series
s1 = pd.Series(df2['area'].apply(get_area_index).to_list()) # 转换为series
s2 = pd.Series(df2['总价'].str.strip('万元').tolist())
df3 = df2.assign(房型Index=s,areaIndex=s1,总价Index=s2)
# print(df3)

data = df3[['areaIndex', '房型Index', '总价Index']].values.tolist()
# print(data)

data = [[d[0], d[1], d[2]] for d in data]

# print(data)
(
    Bar3D()
    .add(
        series_name="",
        data=data,
        xaxis3d_opts=opts.Axis3DOpts(type_="category", data=area),
        yaxis3d_opts=opts.Axis3DOpts(type_="category", data=chamber),
        zaxis3d_opts=opts.Axis3DOpts(type_="value"),
    )
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(
            max_=20,
            range_color=[
                "#313695",
                "#4575b4",
                "#74add1",
                "#abd9e9",
                "#e0f3f8",
                "#ffffbf",
                "#fee090",
                "#fdae61",
                "#f46d43",
                "#d73027",
                "#a50026",
            ],
        )
    )
    # .render("bar3d_punch_card.html")
).render_notebook()
